# Script to generate networks from the Harry Potter movie scripts

In [1]:
import networkx as nx

In [2]:
script = open("../movieScripts/(1)_Harry_Potter_and_the_Philosophers_Stone.pdf").read()

In [3]:
script

'%PDF-1.4\n%\xc7\xec\x8f\xa2\n5 0 obj\n<</Length 6 0 R/Filter /FlateDecode>>\nstream\nx\x9cmU\xdbR\xe3F\x10}\xf7W4O\xb0U^-\x97\xda@x\xd9\x08\xf0\x82\x83o\xc1"\xd4>\xa5\xc6\xd2H\x9aX\x9a\xd1\xce\x8c\xd0:_\x9f\xd3#q\xf1&\xb8\xb0\xd0\xf4\xf4\xe9\xee\xd3\xa7\x9b\xeft\x1c\x9d\xd01\x7f\x86gZ\x8f>=\x9cS\xe1F\xc7T\x8c\xbe\x8fN\x82\x91\x86GZ\xd3U\x82\x0b\x17tr\x12]\x9c\x7f\xa6$\x1f\x1dG\x17g\xfc\xd3\xdf\xa3_\x8e\xe9\xfc\xfc4\x82\xa9\x1e\x1d\xdd\tkw\xb42\xdeKKBg\xe4KI\xabRU\xc6\x99\xa6\x94\xf6\xd0\xd1\xda\x1b-i\x9dZ\xd5\xf8\x0f\xc9\xdf\xa3\xcf\xbfFg\x0c<\x83\xb7\xb4\x92v\xa6\xa5NU\x15\xe5j\xf0OM\xddT\xd2Kr\xc1\x89rci/P\x9c\xac\xd6\xd1\x87\xc3\xd1K\x8e\x80J\xe0w\xbd\x9c\xaff\x93d2\xa6\xe5bB\xcb\xaf\x14\xd3\xfdtqC\xeb\xeb\x87\xe9*\xa1\xe9\x9a\x16\xcb\'z\\\x1d\x1c\x1cPR*G\xde\x98-\x9d\xd1\xd3\xddr6\xa1L\xec\xf8d\x88\x0e\xf4#/#\xba8\xa6F\x14\xd2\x91\xc9\xa9\xb3\n\xe15\x0e\x1ai\xc7tA\xa5imo\x11>-\x95.H\xd0)\x9d|:\r\x16\xaa\xcd\xb3\x92\xe3@K8aL\x1bb\xf8]\xc3\xb7\x95\xa7\xb6\x89\xe8\xcet\xf2\x99\x11\xa